In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.getOrCreate()

22/01/10 21:13:29 WARN Utils: Your hostname, daniyal resolves to a loopback address: 127.0.1.1; using 192.168.228.38 instead (on interface wlp1s0)
22/01/10 21:13:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/10 21:13:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/10 21:13:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/10 21:13:33 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/01/10 21:13:33 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/01/10 21:13:33 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/01/10 21:13:3

In [3]:
df=spark.read.csv('data.csv',header=True,inferSchema=True)

In [6]:
df.printSchema()

root
 |-- season: double (nullable = true)
 |-- yr: double (nullable = true)
 |-- mnth: double (nullable = true)
 |-- holiday: double (nullable = true)
 |-- weekday: double (nullable = true)
 |-- workingday: double (nullable = true)
 |-- weathersit: double (nullable = true)
 |-- temp: double (nullable = true)
 |-- atemp: double (nullable = true)
 |-- hum: double (nullable = true)
 |-- windspeed: double (nullable = true)
 |-- cnt: double (nullable = true)



In [7]:
df.show()

+------+---+----+-------+-------+----------+----------+--------+--------+--------+---------+------+
|season| yr|mnth|holiday|weekday|workingday|weathersit|    temp|   atemp|     hum|windspeed|   cnt|
+------+---+----+-------+-------+----------+----------+--------+--------+--------+---------+------+
|   1.0|0.0| 1.0|    0.0|    6.0|       0.0|       2.0|0.344167|0.363625|0.805833| 0.160446| 985.0|
|   1.0|0.0| 1.0|    0.0|    0.0|       0.0|       2.0|0.363478|0.353739|0.696087| 0.248539| 801.0|
|   1.0|0.0| 1.0|    0.0|    1.0|       1.0|       1.0|0.196364|0.189405|0.437273| 0.248309|1349.0|
|   1.0|0.0| 1.0|    0.0|    2.0|       1.0|       1.0|     0.2|0.212122|0.590435| 0.160296|1562.0|
|   1.0|0.0| 1.0|    0.0|    3.0|       1.0|       1.0|0.226957| 0.22927|0.436957|   0.1869|1600.0|
|   1.0|0.0| 1.0|    0.0|    4.0|       1.0|       1.0|0.204348|0.233209|0.518261|0.0895652|1606.0|
|   1.0|0.0| 1.0|    0.0|    5.0|       1.0|       2.0|0.196522|0.208839|0.498696| 0.168726|1510.0|


In [8]:
train,test=df.randomSplit([0.7,0.3])

In [9]:
from pyspark.ml.feature import VectorAssembler,VectorIndexer 

In [10]:
from pyspark.ml.regression import GBTRegressor

In [11]:
from pyspark.ml.evaluation import RegressionEvaluator

In [12]:
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder

In [13]:
from pyspark.ml.pipeline import Pipeline

In [14]:
cols = [c for c in df.columns if c!='cnt']

In [15]:
cols

['season',
 'yr',
 'mnth',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed']

In [27]:
va = VectorAssembler(inputCols=cols, outputCol= 'vafeatures')

In [28]:
vi = VectorIndexer(inputCol='vafeatures', outputCol='features', maxCategories=4)

In [29]:
gbt = GBTRegressor(featuresCol='features', labelCol='cnt')

In [30]:
evalualtor = RegressionEvaluator(predictionCol=gbt.getPredictionCol(),
    labelCol=gbt.getLabelCol(),
    metricName='rmse',)

In [31]:
print(gbt.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 

In [32]:
param = ParamGridBuilder().addGrid(gbt.maxIter,[10,15]).addGrid(gbt.stepSize,[0.05,0.08]).addGrid(gbt.maxDepth,[3,10])

In [33]:
cv = CrossValidator( estimator=gbt,
    estimatorParamMaps=param,
    evaluator=evalualtor,
    numFolds=2,
    seed=12,
    parallelism=8)

In [25]:
pipe = Pipeline(stages=[va,vi,cv])

In [26]:
pModel = pipe.fit(train)

TypeError: object of type 'ParamGridBuilder' has no len()

In [ ]:
bestModel= pModel.stages[-1].explainParams

In [ ]:
bestModel

In [ ]:
eval = pModel.transform(test)
eval

In [ ]:
finalRes = evaluator.setMetricName('r2').evaluate(eval)
finalRes #variances explained by the model